In [115]:
import json
import urllib
import glob
import os
import time
import xml.etree.ElementTree as ET

In [116]:
user="aa.annaaniol@gmail.com" 
lpmn="any2txt|wcrft2|liner2({\"model\":\"5nam\"})" 
url="http://ws.clarin-pl.eu/nlprest2/base" 

In [117]:
potop_file = "../lab2/sources/potop.txt"

In [118]:
with open (potop_file, "rb") as myfile:
    doc = myfile.read()
    upload_request = urllib.request.Request(url + '/upload/', doc, {'Content-Type': 'binary/octet-stream'})
    fileid = urllib.request.urlopen(upload_request).read().decode("utf-8") 

In [119]:
data = {'lpmn': lpmn, 'user': user, 'file':fileid}

doc = json.dumps(data)
binary_doc = doc.encode("utf-8")
# print(binary_doc)

In [120]:
task_request = urllib.request.Request(url + '/startTask/', binary_doc, {'Content-Type': 'application/json'})

In [121]:
taskid = urllib.request.urlopen(task_request).read()
taskid = taskid.decode("utf-8")

In [122]:
status_request = urllib.request.Request(url + '/getStatus/' + taskid)

In [124]:
# wait a moment for processing to be done and run

response = urllib.request.urlopen(status_request);
data = json.load(response)
print(data)

{'value': [{'name': 'file', 'fileID': '/requests/liner2/c890707c-b74b-4f1f-a911-b65e805c7d97'}], 'status': 'DONE'}


In [125]:
file_id = data['value'][0]['fileID']
print(file_id)

/requests/liner2/c890707c-b74b-4f1f-a911-b65e805c7d97


In [126]:
content_request = urllib.request.Request(url + '/download' + file_id)

In [127]:
content = urllib.request.urlopen(content_request).read()
content = content.decode("utf-8")

In [129]:
root = ET.fromstring(content)

In [130]:
def print_sentence(sentence):
    for tok in sentence:
        orth = tok.find('orth')
        if orth != None:
            print(orth.text, end=' ')

In [183]:
names = {}
places = {}

In [184]:
def is_name(tok):
    if tok is None:
        return False
    if_name = False
    for ann in tok.findall('ann'):
        if ann != None:
            if ann.text != '0':
                if 'person_first_nam' in ann.attrib['chan']:
                    if_name = True
                elif 'person_last_nam' in ann.attrib['chan']:  
                    if_name = True
    return if_name

def is_place(tok):
    if tok is None:
        return False
    if_place = False
    for ann in tok.findall('ann'):
        if_place = False
        if ann != None:
            if ann.text != '0':
                if 'road_nam' in ann.attrib['chan']:
                    if_place = True
                elif 'city_nam' in ann.attrib['chan']:
                    if_place = True
                elif 'country_nam' in ann.attrib['chan']:
                    if_place = True
    return if_place

In [185]:
### Build a name-contexts and place-contexts dictionary

for chunk in root:
    for sentence in chunk:
        last_tok = None
        last_phrase = ''
        
        for tok in sentence:
            
            if_name = is_name(tok)
            if_place = is_place(tok)
        
            lex = tok.find('lex')

            if lex != None:
                base = lex.find('base')
                if base != None and if_name:
                    if is_name(last_tok):   # processing 2nd or greater part of name
                        last_phrase = last_phrase + ' ' + base.text
                    else:   # processing 1st part of name
                        last_phrase = base.text

                elif is_name(last_tok):
                    phrase = last_phrase
                    if phrase in names:
                        names[phrase].append(sentence)
                    else:
                        names[phrase] = [sentence]
                        
                if base != None and if_place:
                    if is_place(last_tok):   # processing 2nd or greater part of name
                        last_phrase = last_phrase + ' ' + base.text
                    else:   # processing 1st part of name
                        last_phrase = base.text

                elif is_place(last_tok):
                    phrase = last_phrase
                    if phrase in places:
                        places[phrase].append(sentence)
                    else:
                        places[phrase] = [sentence]  
                        
                                    
                last_tok = tok
            
        if is_name(last_tok):
            phrase = last_phrase
            if phrase in names:
                names[phrase].append(sentence)
            else:
                names[phrase] = [sentence]  
                
        if is_place(last_tok):
            phrase = last_phrase
            if phrase in places:
                places[phrase].append(sentence)
            else:
                places[phrase] = [sentence]  

In [186]:
len(names)

930

In [187]:
# print(names.keys())

In [188]:
len(places)

404

In [181]:
# print(places.keys())

In [182]:
def print_all_contexts_for_name(phrase):
    i = 1
    for sentence in names[phrase]:
        print('\n', i, 'occurrence:')
        print_sentence(sentence)
        i += 1
        
def print_all_contexts_for_place(phrase):
    i = 1
    for sentence in places[phrase]:
        print('\n', i, 'occurrence:')
        print_sentence(sentence)
        i += 1

In [174]:
print_all_contexts_for_name('Stanisław')


 1 occurrence:
Zapłonął na to pan Stanisław i odrzekł : 
 2 occurrence:
- To do mnie nie należy ! - odrzekł sucho pan Stanisław . 
 3 occurrence:
Jeździli tedy obaj przed szeregami czyniąc rozkosz oczom swym moderunkiem i postawą ; pan Stanisław , czarny jak żuk na podobieństwo wszystkich Skrzetuskich , z twarzą męską , groźną i ozdobioną długą ukośną blizną od cięcia miecza pozostałą , z kruczą , rozwianą na wiatr brodą ; pan Władysław , tłustawy , z długimi jasnymi wąsami , z odwiniętą wargą dolną i oczyma w czerwonych obwódkach , łagodny i poczciwy , mniej przypominał Marsa , ale niemniej była to szczera dusza żołnierska , jak salamandra w ogniu się kochająca , rycerz znający wojnę jak swoje dziesięć palców i odwagi nieporównanej . 
 4 occurrence:
- Miło mi zabrać znajomość z waćpanem dobrodziejem - odparł Stanisław - tym bardziej że imię znane mi było dobrze , bo je rycerstwo w całej Rzeczypospolitej ze czcią powtarza i za przykład podaje . 
 5 occurrence:
Jan siedział z dłońmi op

In [169]:
print_all_contexts_for_name('Stanisław Skrzetuski')


 1 occurrence:
Nad kaliskimi chłopy dzierżył rotmistrzowską buławę pan Stanisław Skrzetuski , z rodu dzielnych wojowników , stryjeczny Jana , słynnego zbarażczyka . 
 2 occurrence:
Pan Stanisław Skrzetuski , rotmistrz kaliski , a stryjeczny słynnego zbarażczyka Jana , włosy rwał na głowie , bo i jego łanowcy , idąc za przykładem towarzystwa " , poczęli wyciekać " z obozu . 
 3 occurrence:
- Jaśnie wielmożny wojewodo ! - odpowiadał pan Stanisław Skrzetuski . 
 4 occurrence:
O świtaniu nadjechał pan Stanisław Skrzetuski z wieścią , że Szwedzi przyciągnęli już pod Wałcz , o jeden dzień marszu od polskiego obozu . 
 5 occurrence:
Wyprowadził jeszcze pan Stanisław Skrzetuski dwie chorągwie kaliszanów , chcąc śmiałym atakiem zmieszać Szwedów , ale nie poszli ochotnie ; oddział rozciągnął się zaraz w bezładną kupę , bo gdy odważniejsi parli konie naprzód , tchórzliwsi wstrzymywali je umyślnie . 
 6 occurrence:
- Bo ją wojewoda poznański i inni wydali pod Ujściem w ręce nieprzyjaciela - odp

In [170]:
print_all_contexts_for_place('wielki księstwo litewski')


 1 occurrence:
Młodszy jego towarzysz , w długiej peruce i w cudzoziemskim stroju , był to książę Bogusław , stryjeczny Janusza , koniuszy Wielkiego Księstwa Litewskiego . 
 2 occurrence:
Nie mogąc lepiej i dogodniej postąpić w najburzliwszym teraźniejszym rzeczy stanie , po utraceniu wszelkiej nadziei na pomoc najjaśniejszego króla , my , panowie i stany Wielkiego Księstwa Litewskiego , koniecznością zmuszeni , poddajemy się pod protekcję najjaśniejszego króla szwedzkiego na tych warunkach : 
 3 occurrence:
2 / Wielkie Księstwo Litewskie nie będzie do Szwecji wcielone , lecz z nią takim sposobem połączone , jak dotąd z Koroną Polską , to jest , aby naród narodowi , senat senatowi , a rycerstwo rycerstwu we wszystkim było równe . 
 4 occurrence:
Od czasu jak nieśmiertelnej pamięci pan marszałek Wielkiego Księstwa Litewskiego zapisał dobra białostockie na utrzymanie zamku tykocińskiego , ja , będąc tychże dóbr gubernatorem , wiernie i poczciwie wszelkie czynsze na pożytek tegoż zamku 

In [171]:
### Occurrence ranking

In [153]:
sorted_names = sorted(names.items(), key=lambda x: len(x[1]), reverse=True)
sorted_places = sorted(places.items(), key=lambda x: len(x[1]), reverse=True)

In [154]:
print('top 10 occurrrences of names:')
for name in sorted_names[:10]:
    print(name[0], len(name[1]))

top 10 occurrrences of names:
Kmicic 802
Wołodyjowski 587
Zagłoba 498
Andrzej 417
Bogusław 395
Czarniecki 274
Sapieha 193
Miller 182
Jan Kazimierz 165
Michał 149


In [155]:
print('top 10 occurrrences of places:')
for place in sorted_places[:10]:
    print(place[0], len(place[1]))

top 10 occurrrences of places:
Litwa 88
rzeczpospolita 78
Warszawa 77
Częstochowa 68
Zamość 49
Kiejdanach 48
Taurogach 44
Szwecja 31
Zagłoba 29
Niemcy 27
